In [53]:
import pandas as pd

In [2]:
df = pd.DataFrame()

fiscl_years = [2020, 2021, 2022, 2023  ,2024    ,   2025,2026]

for year in fiscl_years:
    df_year = pd.read_parquet(f'cleaned_budget_{year}.parquet')
    df = pd.concat([df, df_year], ignore_index=True)

# Display the first few rows of the DataFrame
pd.set_option('display.max_columns', None)
df.head()

,budget_id,budget_type,fiscal_year,budget_amount,budget_description,funding_code,funding_source,department_code,department_name,abbreviation,agency_code,full_agency_code,agency_name,org_code,org_name,region_code,region_description,prexc_fpap_id,uacs_object_code,uacs_classification,uacs_sub_class,uacs_group,uacs_object_name,uacs_sub_object_name
0,GAA-2020-0000000001,GAA,2020,0.0,General Administration and Support,nan,None,None,None,None,None,None,None,nan,None,nan,None,100000000000000,nan,None,None,None,None,None
1,GAA-2020-0000000002,GAA,2020,543636.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010101001,Expenses,Personnel Services,Salaries and Wages,Salaries and Wages - Regular,Basic Salary - Civilian
2,GAA-2020-0000000003,GAA,2020,60383.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010102000,Expenses,Personnel Services,Salaries and Wages,Salaries and Wages - Casual/Contractual,Salaries and Wages - Casual/Contractual
3,GAA-2020-0000000004,GAA,2020,16824.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010201001,Expenses,Personnel Services,Other Compensation,Personal Economic Relief Allowance (PERA),PERA - Civilian
4,GAA-2020-0000000005,GAA,2020,11772.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010202000,Expenses,Personnel Services,Other Compensation,Representation Allowance (RA),Representation Allowance (RA)


In [37]:
# Separate NEP and GAA
nep = df[df['budget_type'] == 'NEP']
gaa = df[df['budget_type'] == 'GAA']

In [38]:
# Aggregate to comparable level
group_cols = [
    'fiscal_year',
    'department_code',
    'department_name',
    'full_agency_code',
    'agency_name',
    'region_code',
    'region_description',
    'uacs_object_code',
    'uacs_sub_object_name'
]

In [39]:
nep_agg = nep.groupby(group_cols)['budget_amount'].sum().reset_index()
gaa_agg = gaa.groupby(group_cols)['budget_amount'].sum().reset_index()

# Merge NEP and GAA
merged = nep_agg.merge(
    gaa_agg,
    on=group_cols,
    how='inner',
    suffixes=('_nep', '_gaa')
)

In [6]:
# merged.count()

In [40]:
# Calculate drift
merged['abs_change'] = merged['budget_amount_gaa'] - merged['budget_amount_nep']
merged['pct_change'] = merged['abs_change'] / merged['budget_amount_nep'].replace(0, 1)

In [41]:
merged[merged['pct_change']>0]

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,budget_amount_gaa,abs_change,pct_change
19,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,640261.0,720261.0,80000.0,0.124949
21,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5020101000,Traveling Expenses - Local,313454.0,338454.0,25000.0,0.079757
25,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5020301002,Office Supplies Expenses,36657.0,38657.0,2000.0,0.054560
46,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5021003000,Extraordinary and Miscellaneous Expenses,185442.0,260442.0,75000.0,0.404439
76,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5029905001,Rents - Building and Structures,217534.0,226534.0,9000.0,0.041373
...,...,...,...,...,...,...,...,...,...,...,...,...,...
299798,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5021499000,Subsidies - Others,1038000.0,1188000.0,150000.0,0.144509
299806,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5029905001,Rents - Building and Structures,278211.0,288211.0,10000.0,0.035944
299815,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5029999099,Other Maintenance and Operating Expenses,692380.0,736527.0,44147.0,0.063761
299816,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5060404001,Buildings,0.0,5000.0,5000.0,5000.000000


In [42]:
merged['anomaly_threshold'] = (
    (merged['pct_change'].abs() > 0.3) |
    (merged['abs_change'].abs() > 50_000_000)
)

In [43]:
merged[merged['anomaly_threshold']]

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,budget_amount_gaa,abs_change,pct_change,anomaly_threshold
46,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5021003000,Extraordinary and Miscellaneous Expenses,185442.0,260442.0,75000.0,0.404439,True
83,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060404001,Buildings,2877000.0,4377000.0,1500000.0,0.521376,True
91,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060405099,Other Machinery and Equipment,0.0,34600.0,34600.0,34600.000000,True
92,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060406001,Motor Vehicles,0.0,20000.0,20000.0,20000.000000,True
94,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060407001,Furniture and Fixtures,0.0,2500.0,2500.0,2500.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298921,2025,39,Department of Human Settlements and Urban Deve...,39001,Office of the Secretary,13,National Capital Region (NCR),5029904000,Transportation and Delivery Expenses,4169.0,1893.0,-2276.0,-0.545934,True
298922,2025,39,Department of Human Settlements and Urban Deve...,39001,Office of the Secretary,13,National Capital Region (NCR),5029905001,Rents - Building and Structures,12084.0,5257.0,-6827.0,-0.564962,True
298926,2025,39,Department of Human Settlements and Urban Deve...,39001,Office of the Secretary,13,National Capital Region (NCR),5029999099,Other Maintenance and Operating Expenses,4547.0,8864.0,4317.0,0.949417,True
299816,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5060404001,Buildings,0.0,5000.0,5000.0,5000.000000,True


In [44]:
merged['z_score'] = merged.groupby(
    ['fiscal_year', 'uacs_object_code']
)['pct_change'].transform(
    lambda x: (x - x.mean()) / x.std(ddof=0)
)

merged['anomaly_zscore'] = merged['z_score'].abs() > 2
merged[merged['anomaly_zscore']]

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,budget_amount_gaa,abs_change,pct_change,anomaly_threshold,z_score,anomaly_zscore
46,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5021003000,Extraordinary and Miscellaneous Expenses,185442.0,260442.0,75000.0,0.404439,True,7.649304,True
92,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060406001,Motor Vehicles,0.0,20000.0,20000.0,20000.000000,True,4.645261,True
96,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060409099,"Other Property, Plant and Equipment",0.0,15900.0,15900.0,15900.000000,True,9.085265,True
98,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060699000,Other Intangible Assets,0.0,12000.0,12000.0,12000.000000,True,5.401966,True
123,2020,01,Congress of the Philippines (CONGRESS),01002,Senate Electoral Tribunal,13,National Capital Region (NCR),5020101000,Traveling Expenses - Local,10.0,710.0,700.0,70.000000,True,28.531562,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297513,2025,38,Department of Transportation (DOTr),38003,Maritime Industry Authority (MARINA),08,Region VIII - Eastern Visayas,5020201002,Training Expenses,154.0,1454.0,1300.0,8.441558,True,14.647586,True
297516,2025,38,Department of Transportation (DOTr),38003,Maritime Industry Authority (MARINA),08,Region VIII - Eastern Visayas,5020302000,Accountable Forms Expenses,5.0,12505.0,12500.0,2500.000000,True,22.383029,True
298282,2025,38,Department of Transportation (DOTr),38006,Philippine Coast Guard,13,National Capital Region (NCR),5060406001,Motor Vehicles,0.0,850000.0,850000.0,850000.000000,True,15.502126,True
298283,2025,38,Department of Transportation (DOTr),38006,Philippine Coast Guard,13,National Capital Region (NCR),5060406004,Watercrafts,45000.0,845000.0,800000.0,17.777778,True,2.236052,True


In [46]:
merged['region_mean'] = merged.groupby(
    ['fiscal_year', 'uacs_object_code']
)['pct_change'].transform('mean')

merged['region_anomaly'] = (
    (merged['pct_change'].abs() > 0.05) &
    ((merged['pct_change'] - merged['region_mean']).abs() > 0.25)
)
merged[merged['region_anomaly'] ]

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,budget_amount_gaa,abs_change,pct_change,anomaly_threshold,z_score,anomaly_zscore,region_mean,region_anomaly
19,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,640261.0,720261.0,80000.0,0.124949,False,-0.316227,False,49370.499539,True
46,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5021003000,Extraordinary and Miscellaneous Expenses,185442.0,260442.0,75000.0,0.404439,True,7.649304,True,-0.002186,True
82,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5029999099,Other Maintenance and Operating Expenses,612155.0,718155.0,106000.0,0.173159,False,-0.062714,False,128.952854,True
83,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060404001,Buildings,2877000.0,4377000.0,1500000.0,0.521376,True,-0.129262,False,2373.967868,True
91,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060405099,Other Machinery and Equipment,0.0,34600.0,34600.0,34600.000000,True,0.066384,False,20010.457454,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298926,2025,39,Department of Human Settlements and Urban Deve...,39001,Office of the Secretary,13,National Capital Region (NCR),5029999099,Other Maintenance and Operating Expenses,4547.0,8864.0,4317.0,0.949417,True,-0.068933,False,39.033838,True
299798,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5021499000,Subsidies - Others,1038000.0,1188000.0,150000.0,0.144509,False,-0.061426,False,1539.144074,True
299815,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5029999099,Other Maintenance and Operating Expenses,692380.0,736527.0,44147.0,0.063761,False,-0.070537,False,39.033838,True
299816,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5060404001,Buildings,0.0,5000.0,5000.0,5000.000000,True,-0.005524,False,5378.196274,True


In [47]:
merged['historical_mean'] = merged.groupby(
    ['full_agency_code', 'region_code', 'uacs_object_code']
)['pct_change'].transform('mean')

merged['historical_std'] = merged.groupby(
    ['full_agency_code', 'region_code', 'uacs_object_code']
)['pct_change'].transform('std')

merged['historical_anomaly'] = (
    (merged['pct_change'] - merged['historical_mean']).abs() >
    2 * merged['historical_std']
)

In [48]:
merged['anomaly_score'] = (
    merged['anomaly_threshold'].astype(int) +
    merged['anomaly_zscore'].astype(int) +
    merged['historical_anomaly'].astype(int) +
    merged['region_anomaly'].astype(int)
)

merged['is_anomaly'] = merged['anomaly_score'] >= 2


In [49]:
def explain(row):
    reasons = []
    if row['anomaly_threshold']:
        reasons.append("large budget change")
    if row['anomaly_zscore']:
        reasons.append("statistically unusual compared to peers")
    if row['historical_anomaly']:
        reasons.append("inconsistent with historical trends")
    if row['region_anomaly']:
        reasons.append("differs from other regions")
    return '' if not reasons else "Flagged due to: " + ", ".join(reasons)

merged['explanation'] = merged.apply(explain, axis=1)

In [50]:
merged[merged['is_anomaly']]

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,budget_amount_gaa,abs_change,pct_change,anomaly_threshold,z_score,anomaly_zscore,region_mean,region_anomaly,historical_mean,historical_std,historical_anomaly,anomaly_score,is_anomaly,explanation
46,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5021003000,Extraordinary and Miscellaneous Expenses,185442.0,260442.0,75000.0,0.404439,True,7.649304,True,-0.002186,True,0.333101,0.192979,False,3,True,"Flagged due to: large budget change, statistic..."
83,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060404001,Buildings,2877000.0,4377000.0,1500000.0,0.521376,True,-0.129262,False,2373.967868,True,0.080808,0.227394,False,2,True,"Flagged due to: large budget change, differs f..."
91,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060405099,Other Machinery and Equipment,0.0,34600.0,34600.0,34600.000000,True,0.066384,False,20010.457454,True,11533.386210,19976.273522,False,2,True,"Flagged due to: large budget change, differs f..."
92,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060406001,Motor Vehicles,0.0,20000.0,20000.0,20000.000000,True,4.645261,True,571.510609,True,6666.666667,11547.005384,False,3,True,"Flagged due to: large budget change, statistic..."
94,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5060407001,Furniture and Fixtures,0.0,2500.0,2500.0,2500.000000,True,0.423007,False,504.052500,True,43192.083333,84673.230688,False,2,True,"Flagged due to: large budget change, differs f..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298921,2025,39,Department of Human Settlements and Urban Deve...,39001,Office of the Secretary,13,National Capital Region (NCR),5029904000,Transportation and Delivery Expenses,4169.0,1893.0,-2276.0,-0.545934,True,-0.579081,False,0.054445,True,-0.109187,0.244149,False,2,True,"Flagged due to: large budget change, differs f..."
298922,2025,39,Department of Human Settlements and Urban Deve...,39001,Office of the Secretary,13,National Capital Region (NCR),5029905001,Rents - Building and Structures,12084.0,5257.0,-6827.0,-0.564962,True,-0.567211,False,0.065506,True,-0.141240,0.282481,False,2,True,"Flagged due to: large budget change, differs f..."
298926,2025,39,Department of Human Settlements and Urban Deve...,39001,Office of the Secretary,13,National Capital Region (NCR),5029999099,Other Maintenance and Operating Expenses,4547.0,8864.0,4317.0,0.949417,True,-0.068933,False,39.033838,True,0.237354,0.474709,False,2,True,"Flagged due to: large budget change, differs f..."
299816,2025,40,Department of Migrant Workers (DMW),40001,Office of the Secretary,13,National Capital Region (NCR),5060404001,Buildings,0.0,5000.0,5000.0,5000.000000,True,-0.005524,False,5378.196274,True,252500.000000,350017.856687,False,2,True,"Flagged due to: large budget change, differs f..."


,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,budget_amount_gaa,abs_change,pct_change,anomaly_threshold,z_score,anomaly_zscore,region_mean,region_anomaly,historical_mean,historical_std,historical_anomaly,anomaly_score,is_anomaly,explanation
19,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,640261.0,720261.0,80000.0,0.124949,False,-0.316227,False,49370.499539,True,0.297917,0.202037,False,1,False,Flagged due to: differs from other regions
54271,2021,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,640666.0,820666.0,180000.0,0.280958,False,1.732051,False,0.070239,False,0.297917,0.202037,False,0,False,
102710,2022,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,820666.0,940666.0,120000.0,0.146223,False,1.732051,False,0.036556,False,0.297917,0.202037,False,0,False,
151331,2023,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,1021479.0,1171479.0,150000.0,0.146846,False,-0.492128,False,4001.018356,True,0.297917,0.202037,False,1,False,Flagged due to: differs from other regions
198844,2024,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,1021479.0,1518388.0,496909.0,0.486460,True,-0.623577,False,6103.640923,True,0.297917,0.202037,False,2,True,"Flagged due to: large budget change, differs f..."
249201,2025,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010499009,Lump-sum for Personnel Services,1021479.0,1636479.0,615000.0,0.602068,True,-0.064433,False,0.670774,False,0.297917,0.202037,False,1,False,Flagged due to: large budget change


In [ ]:
merged.to_parquet('detected_anomaly.parquet')

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,uacs_object_code,uacs_sub_object_name,budget_amount_nep,budget_amount_gaa,abs_change,pct_change,anomaly_threshold,z_score,anomaly_zscore,region_mean,region_anomaly,historical_mean,historical_std,historical_anomaly,anomaly_score,is_anomaly,explanation
0,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010101001,Basic Salary - Civilian,1385509.0,1385509.0,0.0,0.0,False,0.066239,False,-0.003846,False,0.000000,0.000000,False,0,False,
1,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010102000,Salaries and Wages - Casual/Contractual,60383.0,60383.0,0.0,0.0,False,-0.058418,False,0.502826,False,0.060100,0.093959,False,0,False,
2,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010201001,PERA - Civilian,46272.0,46272.0,0.0,0.0,False,0.069735,False,-0.004258,False,0.001470,0.003600,False,0,False,
3,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010202000,Representation Allowance (RA),31086.0,31086.0,0.0,0.0,False,0.054204,False,-0.003014,False,0.001189,0.002912,False,0,False,
4,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),5010203001,Transportation Allowance (TA),31086.0,31086.0,0.0,0.0,False,0.054154,False,-0.003018,False,0.001193,0.002923,False,0,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299891,2025,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),5029907099,Other Subscription Expenses,6377.0,6377.0,0.0,0.0,False,-0.050287,False,0.521756,False,0.155986,0.270175,False,0,False,
299892,2025,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),5029922000,Bank Transaction Fee,2137.0,2137.0,0.0,0.0,False,NaN,False,0.000000,False,0.000000,NaN,False,0,False,
299893,2025,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),5029999099,Other Maintenance and Operating Expenses,146317.0,146317.0,0.0,0.0,False,-0.070652,False,39.033838,False,-0.317775,0.550402,False,0,False,
299894,2025,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),5060405003,Information and Communication Technology Equip...,34500.0,34500.0,0.0,0.0,False,-0.075148,False,189.707738,False,0.257592,0.364290,False,0,False,
